# Drug Amount Identification
For my HRP, I will be using AI tools to scrape medical notes to identify drug amounts administered to patients. If I can do this successfully, I will use these methods to flag discrepencies between drugs given to patients according to the prescriptions table, and the notes that have been written up on the patients. Flagging discrepencies for closer examination will help hospitals in the following ways:
1. More accurately track inventory
2. Identify common pain points in data entry
3. Build more robust datasets to work with (less bad data)
4. Treat patients more effectively (more accurate information on treatment a patient has already received)
5. Identify potential fraud

## Setup and Data Exploration
Create the database connection and identify common prescriptions so that we can work with a subset of the data and determine the best model.

In [ ]:
import pandas as pd
import mysql.connector
import yaml
import re
from collections import defaultdict
import spacy
import medspacy
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from gpt4all import GPT4All
from pathlib import Path
import spacy
from spacy.pipeline import EntityRuler
import re
import os
from typing import List, Tuple, Any, Optional
import ast

In [ ]:
# load config file and connect to MySQL
with open("config.yaml", 'r') as f:
    config = yaml.safe_load(f)

conn = mysql.connector.connect(
    host=config["mysql"]["host"],
    user=config["mysql"]["user"],
    password=config["mysql"]["password"],
    port=config["mysql"].get("port", 3306)
)
conn.database = "healthcare_db"

In [ ]:
def execute_query(query):
    try:    
        conn.consume_results()
    except:
        pass
    
    cursor = conn.cursor(buffered=True)
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        columns = [i[0] for i in cursor.description]
        df = pd.DataFrame(result, columns=columns)
    finally:
        cursor.close()
    
    return df

In [ ]:
query = """
SELECT
  LOWER(COALESCE(DRUG_NAME_GENERIC, DRUG)) AS drug_name,
  COUNT(*) AS prescription_count,
  COUNT(DISTINCT SUBJECT_ID) AS patient_count
FROM prescriptions
GROUP BY drug_name
ORDER BY prescription_count DESC
LIMIT 10;
"""

prescription_count_df = execute_query(query)
prescription_count_df

We will focus on these top drugs today. We will also need to map different names for these drugs so that we will be able to deal with hospital employees using different names for these drugs.

In [ ]:
# query to see where raw != generic but generic is not null. Pairs should be unique.
query = """
SELECT
    LOWER(DRUG) AS drug_name,
    LOWER(COALESCE(DRUG_NAME_GENERIC, DRUG)) AS generic_name,
    DRUG AS raw_name
FROM prescriptions
GROUP BY drug_name, generic_name, raw_name
ORDER BY drug_name;
"""

drug_name_mapping_df = execute_query(query)

In [ ]:
# find the top generic name with the most raw names
top_generic_name = drug_name_mapping_df.groupby('generic_name').size().idxmax()
top_raw_names = drug_name_mapping_df[drug_name_mapping_df['generic_name'] == top_generic_name]['raw_name'].tolist()
print(f"Top generic name: {top_generic_name}")
print(f"Raw names for {top_generic_name}: {', '.join(top_raw_names)}")

In [ ]:
# 1) No two rows share the same raw_name
print(drug_name_mapping_df['raw_name'].nunique() == len(drug_name_mapping_df))

# 2) No missing values in either column
print(drug_name_mapping_df['raw_name'].notna().all() and drug_name_mapping_df['generic_name'].notna().all())

# 3) Each raw_name maps to exactly one generic_name
print((drug_name_mapping_df.groupby('raw_name')['generic_name']
          .nunique() == 1).all())

In [ ]:
counts = drug_name_mapping_df.groupby('raw_name')['generic_name'].nunique()

bad_raws = counts[counts > 1].index
bad_cases = (
    drug_name_mapping_df
      .loc[drug_name_mapping_df['raw_name'].isin(bad_raws)]
      .groupby('raw_name')['generic_name']
      .apply(list)
      .reset_index(name='generic_names')
)

print(bad_cases.head(10))

Ok, these many to many relationships are going to be a problem. Today we will ignore because we are going to focus on a subset of the drugs and ensure we have a model that can extract those and their amounts, but in the future we are going to have an issue that will have to be dealt with in the HRP. Hopefully we can get context clues for some and hopefully notes will largely specify say, which type of acetaminophen was administered, or give context clues that we can make a reasonable guess. If not, maybe it makes sense to just flag these as notation was not specific enough.

In [ ]:
# find only the raw_names that map to exactly one generic
many_to_one_raws = (
    drug_name_mapping_df
      .groupby('raw_name')['generic_name']
      .nunique()
      .loc[lambda s: s == 1]
      .index
      .tolist()
)

prescriptions = ", ".join(f"'{r}'" for r in many_to_one_raws)

# 3) Query for the top 10 generics among those raws
query = f"""
SELECT
  LOWER(COALESCE(DRUG_NAME_GENERIC, DRUG)) AS generic_name,
  COUNT(*) AS prescription_count
FROM prescriptions
WHERE LOWER(DRUG) IN ({prescriptions})
GROUP BY generic_name
ORDER BY prescription_count DESC
LIMIT 10;
"""

most_common_generics_df = execute_query(query)
most_common_generics_df

In [ ]:
raw_to_generic = dict(
    zip(
        drug_name_mapping_df['raw_name'],
        drug_name_mapping_df['generic_name']
    )
)

# for every raw name in the dictionary, add the lowercase version mapping to the same generic
for raw_name in list(raw_to_generic.keys()):
    generic_name = raw_to_generic[raw_name]
    raw_to_generic[raw_name.lower()] = generic_name
    raw_to_generic[raw_name.upper()] = generic_name

# Quick sanity check: look at the first 10 mappings
for raw, gen in list(raw_to_generic.items())[:10]:
    print(f"{raw} --> {gen}")

In [ ]:
# query unique categories
query = """
SELECT DISTINCT category
FROM noteevents
WHERE category IS NOT NULL AND category != '';
"""

execute_query(query)

Now we take 8 of our top 10 generics and query the notes that contain any of the raws, or the generic itself, of that generic. Why only 8? sw and ns will show up as part of a lot of words and be hard to search for. We will do one for the categories nursing, general, discharge, physician, pharmacy, and case management to quickly decide a category to focus on.

In [ ]:
def build_query(or_squence, category):
  return f"""
    SELECT
      SUBJECT_ID,
      HADM_ID,
      CHARTDATE,
      CATEGORY,
      TEXT
    FROM NOTEEVENTS
    WHERE LOWER(TEXT) REGEXP '{or_squence}'
    AND CATEGORY = '{category}'
    LIMIT 1;
    """

# drop sw and ns from the top generics
most_common_generics_df = most_common_generics_df[
    ~most_common_generics_df['generic_name'].isin(['sw', 'ns'])
]
top_gens = most_common_generics_df['generic_name'].tolist()

search_terms = {
    raw
    for raw, gen in raw_to_generic.items()
    if gen in top_gens
} | set(top_gens)

or_squence = "|".join(re.escape(term) for term in search_terms)

notes_df = execute_query(build_query(or_squence, 'discharge summary'))
print(f"Found {len(notes_df)} matching notes.")

In [ ]:
def find_top10_drugs(text):
    text_lower = text.lower()
    found = {
        raw_to_generic.get(term, term)
        for term in search_terms
        if term in text_lower
    }
    return list(found)

def print_first_note(notes_df):
    if not notes_df.empty:
        first_note = notes_df.iloc[0]
        print(f"Top 10 drugs in note: {first_note['top10_drugs_in_note']}")
        print(f"First note text: {first_note['TEXT']}")
    else:
        print("No notes found.")

notes_df['top10_drugs_in_note'] = notes_df['TEXT'].apply(find_top10_drugs)

print_first_note(notes_df)

In [ ]:
notes_df = execute_query(build_query(or_squence, 'Nursing/other'))
notes_df['top10_drugs_in_note'] = notes_df['TEXT'].apply(find_top10_drugs)
print_first_note(notes_df)

In [ ]:
notes_df = execute_query(build_query(or_squence, 'Physician'))
notes_df['top10_drugs_in_note'] = notes_df['TEXT'].apply(find_top10_drugs)

print_first_note(notes_df)

In [ ]:
notes_df = execute_query(build_query(or_squence, 'General'))
notes_df['top10_drugs_in_note'] = notes_df['TEXT'].apply(find_top10_drugs)

print_first_note(notes_df)

In [ ]:
notes_df = execute_query(build_query(or_squence, 'Pharmacy'))
notes_df['top10_drugs_in_note'] = notes_df['TEXT'].apply(find_top10_drugs)

print_first_note(notes_df)

In [ ]:
notes_df = execute_query(build_query(or_squence, 'Case Management'))
notes_df['top10_drugs_in_note'] = notes_df['TEXT'].apply(find_top10_drugs)

print_first_note(notes_df)

Using pharmacy notes, they are the clearest and look at a single drug.

In [ ]:
query = f"""
SELECT
    TEXT
FROM NOTEEVENTS
WHERE LOWER(TEXT) REGEXP '{or_squence}'
AND CATEGORY = 'Pharmacy'
LIMIT 1000;
"""

notes_df = execute_query(query)
print(f"Found {len(notes_df)} pharmacy notes matching the search terms.")

notes_df['top10_drugs_in_note'] = notes_df['TEXT'].apply(find_top10_drugs)

## Extraction:
### Regex
First, were gonna do some standard scraping. This is unlikely to work well, but it will provide a baseline that we will aim to beat.

In [ ]:
# build generic to set of aliases map
generic_aliases = defaultdict(set)
for raw, gen in raw_to_generic.items():
    generic_aliases[gen].add(raw)
for gen in list(generic_aliases):
    generic_aliases[gen].add(gen)

# common dosage patterns regex
unit_pattern   = r'(?:mg|g|mcg|μg|units|puffs)'
number_pattern = r'(\d+(?:\.\d+)?)'

def extract_with_aliases(text, drug_list):
    text_lower = text.lower()
    extractions = []
    for gen in drug_list:
        for alias in generic_aliases[gen]:
            esc = re.escape(alias)
            p1 = re.compile(fr'{esc}.{{0,20}}?{number_pattern}\s*{unit_pattern}', re.IGNORECASE)
            p2 = re.compile(fr'{number_pattern}\s*{unit_pattern}.{{0,20}}?{esc}', re.IGNORECASE)
            for pat in (p1, p2):
                for m in pat.finditer(text_lower):
                    extractions.append({
                        m.group(0).strip()
                    })
    return extractions

notes_df['dosage_extractions'] = notes_df.apply(
    lambda row: extract_with_aliases(row['TEXT'], row['top10_drugs_in_note']),
    axis=1
)

In [ ]:
print("Note 0")
print(notes_df.iloc[0]['top10_drugs_in_note'])
print(f"\nExtractions: {notes_df.iloc[0]['dosage_extractions']}\n")
print(notes_df.iloc[0]['TEXT'])

In [ ]:
print("Note 1")
print(notes_df.iloc[1]['top10_drugs_in_note'])
print(f"\nExtractions: {notes_df.iloc[1]['dosage_extractions']}\n")
print(notes_df.iloc[1]['TEXT'])

In [ ]:
print("Note 2")
print(notes_df.iloc[2]['top10_drugs_in_note'])
print(f"\nExtractions: {notes_df.iloc[2]['dosage_extractions']}\n")
print(notes_df.iloc[2]['TEXT'])

I'm actually pretty impressed with how well this has worked, but it is not very robust for several reasons:
1. Using a window to extract a number will fail in cases where language is more complex and we are counting on getting lucky with the regex, our code may pick up the wrong number or no number at all
2. We are able to get units, but something like 1000mg is very different from 1000mg hourly for 24 hours. For this to be useful for our goal, we need to be able to get these aggregations
3. We have multiple differing extractions for certain entites, with no way to differentiate between them, crippling our ability to actually develop an automated system

### Medspacy
We need to improve upon this regex model of scraping. So lets use some nlp tools. I will be using medspacy to, using NER, to hopefully better understand this data and get more comprehensive extraction.

In [ ]:
nlp = spacy.load("en_core_web_sm")
ruler = nlp.add_pipe(
    "entity_ruler",
    before="ner",
    config={"phrase_matcher_attr": "LOWER"}
)

patterns = [{"label": "DRUG", "pattern": raw} for raw in raw_to_generic]
ruler.add_patterns(patterns)

unit_pat = r'(?:mg|g|mcg|μg|units|puffs)'
num_pat  = r'(\d+(?:\.\d+)?)'
dosage_re = re.compile(fr'{num_pat}\s*{unit_pat}', re.IGNORECASE)
freq_re   = re.compile(
    r'\b(?:hourly|daily|bid|tid|qid|q\d+h|every\s+\d+\s+(?:hours?|days?))\b',
    re.IGNORECASE
)

In [ ]:
def extract_spacy(text):
    doc = nlp(text)
    meds = []
    for ent in doc.ents:
        if ent.label_ != "DRUG":
            continue
        span = ent.text
        generic = raw_to_generic.get(span.lower(), span.lower())
        if generic not in most_common_generics_df['generic_name'].tolist():
            continue
        
        # still need a sliding window around the span for regex
        start = max(ent.start_char - 50, 0)
        end   = min(ent.end_char + 50, len(text))
        window = text[start:end]
        
        dose_m = dosage_re.search(window)
        if not dose_m:
            continue
        
        freq_m = freq_re.search(window)
        meds.append({
            "generic":   generic,
            "matched":   span,
            "strength":  dose_m.group(0),
            "frequency": freq_m.group(0) if freq_m else None
        })
    return meds

In [ ]:
notes_df['spacy_meds'] = notes_df['TEXT'].apply(extract_spacy)

num = notes_df['spacy_meds'].apply(bool).sum()
print(f"Extracted meds from {num}/{len(notes_df)} notes")
for _, row in notes_df.head(5).iterrows():
    print(row['spacy_meds'])

I'm dissapointed with how poorly this has worked. Compared to the last version, its outputs are a little better when they occur, but there are less attempts at an answer. Compared to our plain regex, we still have the sliding window and regex reliance which is not good, we still have the issue with repeat, differing drug outputs. The only improvement is that we sometimes have frequencies. This is not worth taking over the plain regex solution as is.

### Llama
The final model I will try will be a Llama model downloaded using gpt4all. This will have hardware performance issues compared to the last 2, but it will have far better memory and understanding across a text and should be able to far better label the drugs. We will create a reusable prompt and feed it in with the texts.

In [ ]:
# model_dir = Path(r"D:\gpt4all\models")  
# model_dir.mkdir(exist_ok=True)

# model = GPT4All(
#     model_name="Meta-Llama-3-8B-Instruct.Q4_0.gguf",
#     model_path=model_dir,
#     allow_download=True,
#     n_threads=4
# )

In [ ]:
def find_raws_in_text(generics, text):
    text_lower = text.lower()
    raws = []
    for gen in generics:
        for raw in generic_aliases.get(gen, []):
            if raw in text_lower:
                raws.append(raw)
    return raws

In [ ]:
notes_df['raws_in_note'] = notes_df.apply(
    lambda row: find_raws_in_text(row['top10_drugs_in_note'], row['TEXT']),
    axis=1
)
notes_df.head(1)

In [ ]:
# def make_prompt(target_words, text):
#     prompt = (
#         "You are a medical expert specializing in drug dosage extraction. "
#         "Your task is to identify and extract the dosage information for the following drugs: "
#         f"{', '.join(target_words)}.\n\n"
#         "For each drug, provide the dosage in the format:\n"
#         "drug_name,dosage_value,unit,frequency\n"
#         "If a field is not present in the text, use 'None' for that field.\n\n"
#         "Here is the text to analyze:\n"
#         f"{text}\n\n"
#         "Return one line per drug."
#     )
#     return prompt

# def extract_with_gpt4all(row, max_tokens=256):
#     drugs = row["raws_in_note"]
#     if not drugs:
#         return ""
#     prompt = make_prompt(drugs, row["TEXT"])

#     response = model.generate(prompt, n_predict=max_tokens)
#     return response.strip()

#### No Memory, no gpu, shit computer
I was not able to run this on my machine, so in Supplement_LLM.ipynb which is also attached with this assignment you can see how I got the prompted outputs from Llama. I ran supplement_LLM with a GPU in colab and copy and pasted the outputs below.

In [ ]:
# From GPU enabled environment: Supplement_LLM.ipynb
output1 = "(vancomycin,1000 mg,q48h)"
output2 = "(methadone,20mg,q6h), (fentanyl,350mcg/hr,), (midazolam,10mg/hr,), (propofol,75mcg/kg/min,), (lorazepam,2mg,q4hr),(lorazepam,1mg,q4prn)"
output3 = "(vancomycin,1 gram,q48h), (vancomycin,1 gram,PRN),(vancomycin,1 gram,when level <20 mcg/mL),(vancomycin,hold dose when level >20 mcg/mL)"

In [ ]:
note = notes_df.iloc[0]
print(f"Drugs in note: {note['raws_in_note']}")
print(output1)
print(f"text: {note['TEXT']}")

In [ ]:
note = notes_df.iloc[1]
print(f"Drugs in note: {note['raws_in_note']}")
print(output2)
print(f"text: {note['TEXT']}")

In [ ]:
note = notes_df.iloc[2]
print(f"Drugs in note: {note['raws_in_note']}")
print(output2)
print(f"text: {note['TEXT']}")

This is by far the best of the options. It is smart enough to not output a bunch of copies of the same drug unless there is different for frequency or dosage and does a great job, to my eyes, of picking everything up accuracy. Also, there is no reliance on regex and sliding windows, which makes this method far more robust and transferrable to different types of documents and new drugs. Interestingly, the second option added correct information for drugs that weren't even in our list, which does tell me that I need to prompt more carefully, but also shows the model's potential to be generalized to my larger project.

## Citation

@misc{gpt4all,
  author = {Yuvanesh Anand and Zach Nussbaum and Brandon Duderstadt and Benjamin Schmidt and Andriy Mulyar},
  title = {GPT4All: Training an Assistant-style Chatbot with Large Scale Data Distillation from GPT-3.5-Turbo},
  year = {2023},
  publisher = {GitHub},
  journal = {GitHub repository},
  howpublished = {\url{https://github.com/nomic-ai/gpt4all}},
}

## Flagging
Now that we know what the best solution is for this extraction, we need to use this solution on all pharmacy notes with corresponding prescription entries so that we can make comparisons and begin flagging. For the real thing, I will take 1000 pharmacy noteevents and their corresponding prescription events, feed the notes into the pharmacy table, and compare extracted values from notes to prescription table.

In [ ]:
query_all_pharmacy_notes_and_perscriptions = """
WITH pharmacy_notes AS (
  SELECT *
  FROM NOTEEVENTS
  WHERE CATEGORY = 'Pharmacy'
  AND TEXT IS NOT NULL
  LIMIT 1000
)

SELECT
  n.ROW_ID AS note_row_id,
  n.SUBJECT_ID,
  n.HADM_ID,
  n.CHARTDATE,
  n.TEXT AS note_text,
  p.ROW_ID AS prescription_row_id,
  p.DRUG,
  p.DRUG_NAME_GENERIC,
  p.DRUG_TYPE,
  p.PROD_STRENGTH,
  p.DOSE_VAL_RX,
  p.DOSE_UNIT_RX,
  p.STARTDATE,
  p.ENDDATE
FROM pharmacy_notes n
LEFT JOIN PRESCRIPTIONS p
  ON n.SUBJECT_ID = p.SUBJECT_ID
  AND n.HADM_ID = p.HADM_ID
ORDER BY n.ROW_ID;"""

full_table = execute_query(query_all_pharmacy_notes_and_perscriptions)
full_table.head(5)

In [ ]:
full_table.info()

In [ ]:
# save to csv
df_unique = full_table.drop_duplicates(subset='note_row_id')[['note_row_id', 'note_text']].copy()
df_unique['note_text'] = df_unique['note_text'].str.replace('"', "'", regex=False)
df_unique['note_text'] = '"' + df_unique['note_text'] + '"'
df_unique.to_csv('unique_pharmacy_notes.csv', index=False)

Now read our extractions and get lists of tuples entered into a dict. To see the code for prompting the LLM, see HRP_Run_LLM.ipynb.

In [ ]:
def clean_token(token):
    if token is None:
        return None
    token = str(token).strip().strip('"').strip("'")
    if token == "" or token.lower() == "none":
        return None
    return token

def extract_tuples_fuzzy(text: str):
    """
    Extract tuples like (drug, dose, freq, ...) from free-form LLM text.
    Works even if strings aren't quoted and multiple blocks exist.
    """
    if not isinstance(text, str):
        return []
    
    tuples_raw = re.findall(r"\(([^()]+)\)", text, flags=re.MULTILINE)

    results = []
    for t in tuples_raw:
        parts = [p.strip() for p in t.split(",")]
        if len(parts) > 4:
            parts = parts[:3] + [",".join(parts[3:])]
        while len(parts) < 4:
            parts.append(None)

        parts = [clean_token(p) for p in parts]
        if any(p is not None for p in parts):
            results.append(tuple(parts))

    return results

In [ ]:
df = pd.read_csv("extracted_pharmacy_notes.csv")

row_to_extraction = {}
for _, row in df.iterrows():
    note_id = row["note_row_id"]
    ext_text = row.get("extraction", "")
    parsed = extract_tuples_fuzzy(ext_text)
    row_to_extraction[note_id] = parsed

df["extraction_parsed"] = df["extraction"].apply(extract_tuples_fuzzy)
print(row_to_extraction)

Now we have our extractions, lets flag rows with missing data

In [ ]:
_MASS_UNITS = {
    "mcg": "mcg", "μg": "mcg", "ug": "mcg", "microgram": "mcg", "micrograms": "mcg",
    "mg": "mg", "milligram": "mg", "milligrams": "mg",
    "g": "g", "gm": "g", "gram": "g", "grams": "g"
}

def parse_mass(text):
    """Extract (value_float, unit_canonical) from a string like '50 mcg/hr'.
       Ignores frequency/rate. Returns (None, None) if not found."""
    if not isinstance(text, str):
        return None, None
    s = text.strip().lower()
    m = re.search(r'(\d+(?:\.\d+)?)', s)
    if not m:
        return None, None
    val = float(m.group(1))
    after = s[m.end():]
    for tok in re.findall(r'[a-z]+', after):
        if tok in _MASS_UNITS:
            return val, _MASS_UNITS[tok]
    before = s[:m.start()]
    for tok in reversed(re.findall(r'[a-z]+', before)):
        if tok in _MASS_UNITS:
            return val, _MASS_UNITS[tok]
    return None, None

DRUG_SYNONYMS = {"versed": "midazolam", "zyprexa": "olanzapine"}

def norm_str(s):
    return str(s).strip().lower() if isinstance(s, str) else None

def canonical_drug(name):
    n = norm_str(name)
    return DRUG_SYNONYMS.get(n, n)

def normalize_text(s: str) -> str:
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", " ", s)
    return f" {s.strip()} "

def drug_matches(drug_can: str, rx_row) -> bool:
    """Exact canonical OR word-boundary substring match in DRUG/DRUG_NAME_GENERIC."""
    if not drug_can:
        return False
    if rx_row.get("drug_norm") == drug_can:
        return True
    if rx_row.get("drug_generic_norm") == drug_can:
        return True
    needle = f" {drug_can} "
    for col in ("DRUG", "DRUG_NAME_GENERIC"):
        txt = rx_row.get(col)
        if isinstance(txt, str) and needle in normalize_text(txt):
            return True
    return False

def doses_equal(val_a, unit_a, val_b, unit_b, tol=1e-6):
    """Require same mass unit + numeric equality (tiny tolerance)."""
    if unit_a is None or unit_b is None:
        return False
    if unit_a != unit_b:
        return False
    if val_a is None or val_b is None:
        return False
    return abs(float(val_a) - float(val_b)) <= tol

def ensure_list(obj):
    if isinstance(obj, list):
        return obj
    try:
        import ast
        return ast.literal_eval(obj)
    except Exception:
        return []

In [ ]:
df["extraction_parsed"] = df["extraction_parsed"].apply(ensure_list)
df["note_row_id"] = df["note_row_id"].astype(int)
full_table["note_row_id"] = full_table["note_row_id"].astype(int)

full_table["drug_norm"] = full_table["DRUG"].apply(canonical_drug)
full_table["drug_generic_norm"] = (
    full_table["DRUG_NAME_GENERIC"].apply(canonical_drug)
    if "DRUG_NAME_GENERIC" in full_table.columns else None
)

def unit_from_dose_unit_norm(u):
    """dose_unit_norm may be things like 'mg', 'mcg', or 'mg/hr'. Keep mass unit only."""
    if not isinstance(u, str):
        return None
    _, mass = parse_mass(f"1 {u}")
    return mass

def compute_rx_val_unit_final(row):
    if "rx_val" in row and "rx_unit" in row:
        rv, ru = row["rx_val"], row["rx_unit"]
        if pd.notna(rv) and isinstance(ru, str):
            if ru in _MASS_UNITS:
                return float(rv), ru
            else:
                ru_mass = unit_from_dose_unit_norm(ru)
                if ru_mass:
                    return float(rv), ru_mass

    if "dose_val_norm" in row and "dose_unit_norm" in row:
        dv, du = row["dose_val_norm"], row["dose_unit_norm"]
        if pd.notna(dv) and du is not None:
            du_mass = unit_from_dose_unit_norm(du)
            if du_mass:
                return float(dv), du_mass

    ps = row.get("PROD_STRENGTH")
    vv, uu = parse_mass(ps) if isinstance(ps, str) else (None, None)
    if vv is not None and uu is not None:
        return vv, uu

    vv, uu = parse_mass(row.get("DRUG")) if isinstance(row.get("DRUG"), str) else (None, None)
    if vv is not None and uu is not None:
        return vv, uu

    vv, uu = parse_mass(row.get("DRUG_NAME_GENERIC")) if isinstance(row.get("DRUG_NAME_GENERIC"), str) else (None, None)
    if vv is not None and uu is not None:
        return vv, uu

    return None, None

rx_final = full_table.apply(compute_rx_val_unit_final, axis=1, result_type="reduce")
full_table["rx_val_final"]  = [p[0] for p in rx_final]
full_table["rx_unit_final"] = [p[1] for p in rx_final]

print(
    "Usable Rx dose/units:",
    int(full_table["rx_val"].notna().sum() if "rx_val" in full_table.columns else 0), "(rx_val) ->",
    int(full_table["rx_val_final"].notna().sum()), "(after fallbacks)"
)

rx_by_note = {k: g for k, g in full_table.groupby("note_row_id")}

rows = []
for _, r in df.iterrows():
    nid = int(r["note_row_id"])
    tuples = r.get("extraction_parsed", []) or []
    rxg = rx_by_note.get(nid, pd.DataFrame())

    for idx, t in enumerate(tuples):
        drug = t[0] if len(t) > 0 else None
        dose_raw = t[1] if len(t) > 1 else None

        drug_can = canonical_drug(drug)
        ex_val, ex_unit = parse_mass(dose_raw)

        matched = False
        drug_expected = None
        rx_val_match = None
        rx_unit_match = None

        if drug_can and ex_val is not None and ex_unit is not None and not rxg.empty:
            cand = rxg[rxg.apply(lambda rr: drug_matches(drug_can, rr), axis=1)]
            if not cand.empty:
                for _, rx in cand.iterrows():
                    if doses_equal(ex_val, ex_unit, rx["rx_val_final"], rx["rx_unit_final"]):
                        matched = True
                        drug_expected = (
                            rx["DRUG_NAME_GENERIC"] if isinstance(rx.get("DRUG_NAME_GENERIC"), str)
                            else rx["DRUG"]
                        )
                        rx_val_match = rx["rx_val_final"]
                        rx_unit_match = rx["rx_unit_final"]
                        break

        rows.append({
            "note_row_id": nid,
            "tuple_index": idx,
            "drug_extracted": drug,
            "dose_raw": dose_raw,
            "dose_val_extracted": ex_val,
            "dose_unit_extracted": ex_unit,
            "drug_expected": drug_expected,
            "rx_val_matched": rx_val_match,
            "rx_unit_matched": rx_unit_match,
            "matches_prescription": bool(matched),
        })

results = pd.DataFrame(rows)
matches_df = results[results["matches_prescription"]].reset_index(drop=True)
mismatches_df = results[~results["matches_prescription"]].reset_index(drop=True)

In [ ]:
matches_df.info()

In [ ]:
mismatches_df.info()

In [ ]:
matches_df.head(10)

In [ ]:
mismatches_df.head(10)

In [ ]:
full_table.info()

In [ ]:
full_table[full_table['note_row_id'] == 314570]

Failing because my mysql data is not complete, going back to the roots with PRESCRIPTIONS.csv

In [ ]:
query_pharmacy_notes = """
SELECT
  ROW_ID   AS note_row_id,
  SUBJECT_ID,
  HADM_ID,
  CHARTDATE,
  TEXT     AS note_text
FROM NOTEEVENTS
WHERE CATEGORY = 'Pharmacy'
  AND TEXT IS NOT NULL
ORDER BY ROW_ID
LIMIT 1000;
"""

notes_df = execute_query(query_pharmacy_notes)
print("Notes shape:", notes_df.shape)

def to_key_str(series: pd.Series) -> pd.Series:
    if pd.api.types.is_integer_dtype(series):
        return series.astype("Int64").astype(str)
    return series.astype(str)

notes_df["SUBJECT_ID"] = to_key_str(notes_df["SUBJECT_ID"])
notes_df["HADM_ID"]    = to_key_str(notes_df["HADM_ID"])

notes_pairs = (
    notes_df[["SUBJECT_ID", "HADM_ID"]]
    .dropna()
    .drop_duplicates()
)
pairs_key = set(map(tuple, notes_pairs.values.tolist()))
print("Unique (SUBJECT_ID, HADM_ID) pairs in notes:", len(pairs_key))

usecols = [
    "ROW_ID", "SUBJECT_ID", "HADM_ID",
    "DRUG", "DRUG_NAME_GENERIC", "DRUG_TYPE",
    "PROD_STRENGTH", "DOSE_VAL_RX", "DOSE_UNIT_RX",
    "STARTDATE", "ENDDATE",
]

chunksize = 250_000
kept_chunks = []

reader = pd.read_csv(
    "prescriptions.csv",
    usecols=usecols,
    dtype={"SUBJECT_ID": str, "HADM_ID": str},
    chunksize=chunksize,
    low_memory=True
)

total_rows = 0
kept_rows = 0
for i, chunk in enumerate(reader, start=1):
    total_rows += len(chunk)

    chunk["SUBJECT_ID"] = to_key_str(chunk["SUBJECT_ID"])
    chunk["HADM_ID"]    = to_key_str(chunk["HADM_ID"])

    mask = chunk.apply(lambda r: (r["SUBJECT_ID"], r["HADM_ID"]) in pairs_key, axis=1)
    filtered = chunk[mask]
    kept_rows += len(filtered)

    if not filtered.empty:
        kept_chunks.append(filtered)

    if i % 10 == 0:
        print(f"Processed ~{i * chunksize:,} rows; kept so far: {kept_rows:,}")

rx_df_small = pd.concat(kept_chunks, ignore_index=True) if kept_chunks else pd.DataFrame(columns=usecols)
print("Prescriptions total rows scanned:", f"{total_rows:,}")
print("Prescriptions kept (matching pairs):", rx_df_small.shape)

full_table = notes_df.merge(
    rx_df_small,
    on=["SUBJECT_ID", "HADM_ID"],
    how="left",
    suffixes=("", "_rx")
)

full_table = full_table.rename(columns={"ROW_ID": "prescription_row_id"})

desired_order = [
    "note_row_id", "SUBJECT_ID", "HADM_ID", "CHARTDATE", "note_text",
    "prescription_row_id", "DRUG", "DRUG_NAME_GENERIC", "DRUG_TYPE",
    "PROD_STRENGTH", "DOSE_VAL_RX", "DOSE_UNIT_RX", "STARTDATE", "ENDDATE"
]

cols = [c for c in desired_order if c in full_table.columns] + \
       [c for c in full_table.columns if c not in desired_order]
full_table = full_table[cols]

print("Joined shape:", full_table.shape)
display(full_table.head(10))

In [ ]:
full_table.info()

In [ ]:
df["extraction_parsed"] = df["extraction_parsed"].apply(ensure_list)
df["note_row_id"] = df["note_row_id"].astype(int)
full_table["note_row_id"] = full_table["note_row_id"].astype(int)

full_table["drug_norm"] = full_table["DRUG"].apply(canonical_drug)
full_table["drug_generic_norm"] = (
    full_table["DRUG_NAME_GENERIC"].apply(canonical_drug)
    if "DRUG_NAME_GENERIC" in full_table.columns else None
)

def unit_from_dose_unit_norm(u):
    """dose_unit_norm may be things like 'mg', 'mcg', or 'mg/hr'. Keep mass unit only."""
    if not isinstance(u, str):
        return None
    _, mass = parse_mass(f"1 {u}")
    return mass

def compute_rx_val_unit_final(row):
    if "rx_val" in row and "rx_unit" in row:
        rv, ru = row["rx_val"], row["rx_unit"]
        if pd.notna(rv) and isinstance(ru, str):
            if ru in _MASS_UNITS:
                return float(rv), ru
            else:
                ru_mass = unit_from_dose_unit_norm(ru)
                if ru_mass:
                    return float(rv), ru_mass

    if "dose_val_norm" in row and "dose_unit_norm" in row:
        dv, du = row["dose_val_norm"], row["dose_unit_norm"]
        if pd.notna(dv) and du is not None:
            du_mass = unit_from_dose_unit_norm(du)
            if du_mass:
                return float(dv), du_mass

    ps = row.get("PROD_STRENGTH")
    vv, uu = parse_mass(ps) if isinstance(ps, str) else (None, None)
    if vv is not None and uu is not None:
        return vv, uu

    vv, uu = parse_mass(row.get("DRUG")) if isinstance(row.get("DRUG"), str) else (None, None)
    if vv is not None and uu is not None:
        return vv, uu

    vv, uu = parse_mass(row.get("DRUG_NAME_GENERIC")) if isinstance(row.get("DRUG_NAME_GENERIC"), str) else (None, None)
    if vv is not None and uu is not None:
        return vv, uu

    return None, None

rx_final = full_table.apply(compute_rx_val_unit_final, axis=1, result_type="reduce")
full_table["rx_val_final"]  = [p[0] for p in rx_final]
full_table["rx_unit_final"] = [p[1] for p in rx_final]

print(
    "Usable Rx dose/units:",
    int(full_table["rx_val"].notna().sum() if "rx_val" in full_table.columns else 0), "(rx_val) ->",
    int(full_table["rx_val_final"].notna().sum()), "(after fallbacks)"
)

rx_by_note = {k: g for k, g in full_table.groupby("note_row_id")}

rows = []
for _, r in df.iterrows():
    nid = int(r["note_row_id"])
    tuples = r.get("extraction_parsed", []) or []
    rxg = rx_by_note.get(nid, pd.DataFrame())

    for idx, t in enumerate(tuples):
        drug = t[0] if len(t) > 0 else None
        dose_raw = t[1] if len(t) > 1 else None

        drug_can = canonical_drug(drug)
        ex_val, ex_unit = parse_mass(dose_raw)

        matched = False
        drug_expected = None
        rx_val_match = None
        rx_unit_match = None

        if drug_can and ex_val is not None and ex_unit is not None and not rxg.empty:
            cand = rxg[rxg.apply(lambda rr: drug_matches(drug_can, rr), axis=1)]
            if not cand.empty:
                for _, rx in cand.iterrows():
                    if doses_equal(ex_val, ex_unit, rx["rx_val_final"], rx["rx_unit_final"]):
                        matched = True
                        drug_expected = (
                            rx["DRUG_NAME_GENERIC"] if isinstance(rx.get("DRUG_NAME_GENERIC"), str)
                            else rx["DRUG"]
                        )
                        rx_val_match = rx["rx_val_final"]
                        rx_unit_match = rx["rx_unit_final"]
                        break

        rows.append({
            "note_row_id": nid,
            "tuple_index": idx,
            "drug_extracted": drug,
            "dose_raw": dose_raw,
            "dose_val_extracted": ex_val,
            "dose_unit_extracted": ex_unit,
            "drug_expected": drug_expected,
            "rx_val_matched": rx_val_match,
            "rx_unit_matched": rx_unit_match,
            "matches_prescription": bool(matched),
        })

results = pd.DataFrame(rows)
matches_df = results[results["matches_prescription"]].reset_index(drop=True)
mismatches_df = results[~results["matches_prescription"]].reset_index(drop=True)

In [ ]:
matches_df.info()

In [ ]:
mismatches_df.info()

In [ ]:
matches_df.head(10)

In [ ]:
mismatches_df.head(10)

In [ ]:
mismatches_df_confident = mismatches_df[mismatches_df["drug_expected"].notna()]
mismatches_df_confident.info()

This was a failure. I am not sure the idea is bad, but all of my mismatches are instances where I couldnt get a drug from the prescriptions table at all. This tells me I either have bad data or am not properly matching my extractions to the prescriptions table.

In [ ]:
conn.close() # sad :(